# Aula08.Ex01 - Câmeras - Matrizes Model, View e Projection

### Primeiro, vamos importar as bibliotecas necessárias.

In [1]:
import glfw
from OpenGL.GL import *
import numpy as np
import glm
import math
from numpy import random
from PIL import Image

from shader_s import Shader

### Inicializando janela

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)

altura = 700
largura = 700

window = glfw.create_window(largura, altura, "Programa", None, None)

if (window == None):
    print("Failed to create GLFW window")
    glfwTerminate()
    
glfw.make_context_current(window)



libdecor-gtk-WARNING: Failed to initialize GTK
Failed to load plugin 'libdecor-gtk.so': failed to init


### Constroi e compila os shaders. Também "linka" eles ao programa

#### Novidade aqui: modularização dessa parte do código --- temos agora uma classe e arquivos próprios para os shaders (vs e fs)
Créditos: https://learnopengl.com

In [3]:
ourShader = Shader("vertex_shader.vs", "fragment_shader.fs")
ourShader.use()

program = ourShader.getProgram()

### Preparando dados para enviar a GPU

Até aqui, compilamos nossos Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Modelando um Cubo

Existem diferentes formas de modelar um cubo. Nós usaremos uma estratégia baseada no Quadrado com TRIANGLE_STRIP, conforme vimos na Aula04.Ex05. Assim, um quadrado é modelado usando dois triângulos e precisamos de apenas quatro vértices para isso (devido ao TRIANGLE_STRIP).

Nós usamos 48 vértices para modelar 2 cubos. Cada cubo usa 6 faces de quatro vértices cada.

In [4]:
# preparando espaço para 48 vértices (dois cubos) usando 3 coordenadas (x,y,z)
vertices = np.zeros(48, [("position", np.float32, 3)])

# preenchendo as coordenadas de cada vértice
vertices['position'] = [
    
    ### CUBO 1
    # Face 1 do Cubo 1 (vértices do quadrado)
    (-0.2, -0.2, +0.2),
    (+0.2, -0.2, +0.2),
    (-0.2, +0.2, +0.2),
    (+0.2, +0.2, +0.2),

    # Face 2 do Cubo 1
    (+0.2, -0.2, +0.2),
    (+0.2, -0.2, -0.2),         
    (+0.2, +0.2, +0.2),
    (+0.2, +0.2, -0.2),
    
    # Face 3 do Cubo 1
    (+0.2, -0.2, -0.2),
    (-0.2, -0.2, -0.2),            
    (+0.2, +0.2, -0.2),
    (-0.2, +0.2, -0.2),

    # Face 4 do Cubo 1
    (-0.2, -0.2, -0.2),
    (-0.2, -0.2, +0.2),         
    (-0.2, +0.2, -0.2),
    (-0.2, +0.2, +0.2),

    # Face 5 do Cubo 1
    (-0.2, -0.2, -0.2),
    (+0.2, -0.2, -0.2),         
    (-0.2, -0.2, +0.2),
    (+0.2, -0.2, +0.2),
    
    # Face 6 do Cubo 1
    (-0.2, +0.2, +0.2),
    (+0.2, +0.2, +0.2),           
    (-0.2, +0.2, -0.2),
    (+0.2, +0.2, -0.2),


    #### CUBO 2
    # Face 1 do Cubo 2 (vértices do quadrado)
    (+0.1, +0.1, -0.5),
    (+0.5, +0.1, -0.5),
    (+0.1, +0.5, -0.5),
    (+0.5, +0.5, -0.5),

    # Face 2 do Cubo 2
    (+0.5, +0.1, -0.5),
    (+0.5, +0.1, -0.9),         
    (+0.5, +0.5, -0.5),
    (+0.5, +0.5, -0.9),
    
    # Face 3 do Cubo 2
    (+0.5, +0.1, -0.9),
    (+0.1, +0.1, -0.9),            
    (+0.5, +0.5, -0.9),
    (+0.1, +0.5, -0.9),

    # Face 4 do Cubo 2
    (+0.1, +0.1, -0.9),
    (+0.1, +0.1, -0.5),         
    (+0.1, +0.5, -0.9),
    (+0.1, +0.5, -0.5),

    # Face 5 do Cubo 2
    (+0.1, +0.1, -0.9),
    (+0.5, +0.1, -0.9),         
    (+0.1, +0.1, -0.5),
    (+0.5, +0.1, -0.5),
    
    # Face 6 do Cubo 2
    (+0.1, +0.5, -0.5),
    (+0.5, +0.5, -0.5),           
    (+0.1, +0.5, -0.9),
    (+0.5, +0.5, -0.9)]

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um buffer

In [5]:
buffer_VBO = glGenBuffers(1)

### Enviando coordenadas de vértices para a GPU

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [6]:
# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer_VBO)
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

### Eventos para modificar a posição da câmera.

* Usei as teclas A, S, D e W para movimentação no espaço tridimensional
* Usei a posição do mouse para "direcionar" a câmera

In [7]:
cameraPos   = glm.vec3(0.0,  0.0,  1.0);
cameraFront = glm.vec3(0.0,  0.0, -1.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);

firstMouse = True
yaw   = -90.0	# yaw is initialized to -90.0 degrees since a yaw of 0.0 results in a direction vector pointing to the right so we initially rotate a bit to the left.
pitch =  0.0
lastX =  largura / 2.0
lastY =  altura / 2.0
fov   =  45.0

# timing
deltaTime = 0.0	# time between current frame and last frame
lastFrame = 0.0


firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2


def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp

    if key == glfw.KEY_ESCAPE and action == glfw.PRESS:
        glfw.set_window_should_close(window, True)
    
    cameraSpeed = 1 * deltaTime
    if key == glfw.KEY_W and (action == glfw.PRESS or action == glfw.REPEAT):
        cameraPos += cameraSpeed * cameraFront
    
    if key == glfw.KEY_S and (action == glfw.PRESS or action == glfw.REPEAT):
        cameraPos -= cameraSpeed * cameraFront
    
    if key == glfw.KEY_A and (action == glfw.PRESS or action == glfw.REPEAT):
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == glfw.KEY_D and (action == glfw.PRESS or action == glfw.REPEAT):
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed


def framebuffer_size_callback(window, largura, altura):

    # make sure the viewport matches the new window dimensions note that width and 
    # height will be significantly larger than specified on retina displays.
    glViewport(0, 0, largura, altura)

# glfw: whenever the mouse moves, this callback is called
# -------------------------------------------------------
def mouse_callback(window, xpos, ypos):
    global cameraFront, lastX, lastY, firstMouse, yaw, pitch
   
    if (firstMouse):

        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos # reversed since y-coordinates go from bottom to top
    lastX = xpos
    lastY = ypos

    sensitivity = 0.3 # change this value to your liking
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset
    pitch += yoffset

    # make sure that when pitch is out of bounds, screen doesn't get flipped
    if (pitch > 89.0):
        pitch = 89.0
    if (pitch < -89.0):
        pitch = -89.0

    front = glm.vec3()
    front.x = glm.cos(glm.radians(yaw)) * glm.cos(glm.radians(pitch))
    front.y = glm.sin(glm.radians(pitch))
    front.z = glm.sin(glm.radians(yaw)) * glm.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)

# glfw: whenever the mouse scroll wheel scrolls, this callback is called
# ----------------------------------------------------------------------
def scroll_callback(window, xoffset, yoffset):
    global fov

    fov -= yoffset
    if (fov < 1.0):
        fov = 1.0
    if (fov > 45.0):
        fov = 45.0
    
glfw.set_key_callback(window,key_event)
glfw.set_framebuffer_size_callback(window, framebuffer_size_callback)
glfw.set_cursor_pos_callback(window, mouse_callback)
glfw.set_scroll_callback(window, scroll_callback)

# tell GLFW to capture our mouse
glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED)

### Matrizes Model, View e Projection

Esses são os assuntos dessa e da próxima aula!

#### Novidade: note que paramos de fazer a multiplicação de matrizes de forma explícita. Agora, usamos glm.\<nomeTransformacao> (por ex, glm.translate ou glm.rotate)

In [8]:
def model():
    mat_model = glm.mat4(1.0) # matriz identidade (não aplica transformação!)
    mat_model = np.array(mat_model)    
    return mat_model

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura
    # perspective parameters: fov, aspect, near, far
    mat_projection = glm.perspective(glm.radians(fov), largura/altura, 0.1, 100.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

### Nesse momento, nós exibimos a janela!


In [9]:
glfw.show_window(window)

###  Vamos pegar a localização da variável color (uniform) do Fragment Shader para que possamos alterá-la em nosso laço da janela!

In [10]:
loc_color = glGetUniformLocation(program, "color")

### Loop principal da janela.

In [11]:
glEnable(GL_DEPTH_TEST) ### importante para 3D


def desenha_cubo1():
    # DESENHANDO O CUBO 1 (vértices de 0 até 23)
    for i in range(0,24,4): # incremento de 4 em 4
        R = (i+1)/24
        G = (i+2)/24
        B = (i+3)/24
        glUniform4f(loc_color, R, G, B, 1.0) ### definindo uma cor qualquer com base no i
        glDrawArrays(GL_TRIANGLE_STRIP, i, 4)
    
def desenha_cubo2():
    # DESENHANDO O CUBO 2 (vértices de 24 até 47)
    for i in range(24,48,4): # incremento de 4 em 4
        R = (i+1)/48
        G = (i+2)/48
        B = (i+3)/48
        glUniform4f(loc_color, R, G, B, 1.0) ### definindo uma cor qualquer com base no i
        glDrawArrays(GL_TRIANGLE_STRIP, i, 4)


while not glfw.window_should_close(window):

    currentFrame = glfw.get_time()
    deltaTime = currentFrame - lastFrame
    lastFrame = currentFrame

    glfw.poll_events() 
    
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    

    # computando e enviando matrizes Model, View e Projection para a GPU
    mat_model = model()
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection)    
    

    # desenhando objetos
   
    desenha_cubo1()
    
    desenha_cubo2()

    
    glfw.swap_buffers(window)

glfw.terminate()